In [1]:
import os, sys
import re
import datetime
import rasterio
from rasterio.merge import merge
from rasterio.enums import Resampling
import numpy as np
import glob

acolite_path = '../acolite'
sys.path.append(acolite_path)
import acolite as ac

In [ ]:
path = '****'
tiles = ['S2B_MSIL1C_20230924T103659_N0509_R008_T31SCC_20230924T141949.SAFE',
         'S2B_MSIL1C_20230924T103659_N0509_R008_T31SCD_20230924T141949.SAFE']

tiles_path = [os.path.join(path, tile) for tile in tiles]
output_path = os.path.join(path, 'ACOLITE')

limit_coords = [38.608138, 1.130090, 39.152015, 1.675603]

## ACOLITE settings
settings = {"inputfile":",".join(tiles_path),
            "output":output_path,
            "limit": limit_coords,
            "merge_tiles":True,
            
            ## atmospheric correction algorithms
            "aerosol_correction":"dark_spectrum",
            "dsf_aot_estimate":"tiled",
            ## sunglint correction algorithms
            "dsf_residual_glint_correction":True,

            ## output l2w parameters
            "l2w_parameters":["Rrs_*", "chl_oc3"],

            ## output resolution (S2 only 10, 20, or 60 m) 
            "s2_target_res":10,

            ## increase default L2W masking threshold
            "l2w_mask_threshold":0.05,
            "l2w_mask_negative_rhow":True,

            ## output RGB / L2W maps
            "rgb_rhot":True,
            "rgb_rhos":True,
            "map_l2w":False,

            ## GeoTIFF export options
            "l2r_export_geotiff":True,
            "l2w_export_geotiff":True,
        
            ## Deletion of NetCDF files
            "l1r_delete_netcdf":True,
            "l2r_delete_netcdf":True,
            "l2w_delete_netcdf":True,
        
            ##amount of detail in the terminal outputs
            "verbosity":1}

# It is recommended to use a settings file that only includes the settings you want to change.

In [2]:
def stack_and_clean(input_folder, pattern, output_name, band_name_prefix=None):
    search_path = os.path.join(input_folder, pattern)
    files = sorted(glob.glob(search_path))
    if not files:
        print(f"No se encontraron archivos para el patrón {pattern}")
        return

    src_files = [rasterio.open(f) for f in files]
    bands_data = [src.read(1) for src in src_files]

    stacked = np.stack(bands_data, axis=0)
    meta = src_files[0].meta.copy()
    meta.update(count=len(files))

    # Extraer nombres de bandas (Rrs_443, rhot_833, etc.)
    band_names = []
    for f in files:
        basename = os.path.basename(f)
        match = re.search(rf'{band_name_prefix}_(\d+)', basename)
        if match:
            band_names.append(f"{band_name_prefix}_{match.group(1)}")
        else:
            band_names.append(f"{band_name_prefix}_unk")

    # Extraer fecha
    try:
        basename = os.path.basename(files[0])
        match = re.search(r'(\d{4}_\d{2}_\d{2})', basename)
        acquisition_date = datetime.strptime(match.group(1), "%Y_%m_%d").date().isoformat() if match else "unknown"
    except:
        acquisition_date = "unknown"

    tag_dict = {
        "Product_Level": "L2W" if "_L2W_" in pattern else "L2R",
        "Spectral_Prefix": band_name_prefix or "unknown",
        "Source_Tiles": ', '.join(set([f.split('_T')[1].split('_')[0] for f in files if '_T' in f])),
        "Acquisition_Date": acquisition_date,
        "Resolution": f"{meta['transform'][0]:.2f} m",
        "Bands": ', '.join(band_names),
        "Processed_With": "ACOLITE"
    }

    with rasterio.open(output_name, 'w', **meta) as dst:
        for idx, data in enumerate(stacked):
            dst.write(data, idx + 1)
            dst.set_band_description(idx + 1, band_names[idx])  
        dst.update_tags(**tag_dict)

    # Cierre y limpieza
    for src in src_files:
        src.close()
    for f in files:
        os.remove(f)
    print(f"{output_name} creado con bandas nombradas ({band_names}) y {len(files)} archivos eliminados.")

In [4]:
ac.acolite.acolite_run(settings=settings)

Running ACOLITE processing - Generic GitHub Clone c2025-06-25T10:11:22
Python - linux - 3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]
Platform - Linux 6.8.0-59-generic - x86_64 - #61-Ubuntu SMP PREEMPT_DYNAMIC Fri Apr 11 23:16:11 UTC 2025
Run ID - 20250707_125411
Identified /media/dani/Seagate Basic/data/tests/S2B_MSIL1C_20230924T103659_N0509_R008_T31SCC_20230924T141949.SAFE as Sentinel-2 type
Identified /media/dani/Seagate Basic/data/tests/S2B_MSIL1C_20230924T103659_N0509_R008_T31SCD_20230924T141949.SAFE as Sentinel-2 type


/home/dani/github/WqSat_atcor/notebooks/../acolite/acolite/sentinel2/l1_convert.py:398: RuntimeWarning: Mean of empty slice
  ave_vza = np.nanmean(ave_vza, axis=2)
/home/dani/github/WqSat_atcor/notebooks/../acolite/acolite/sentinel2/l1_convert.py:399: RuntimeWarning: Mean of empty slice
  ave_vaa = np.nanmean(ave_vaa, axis=2)
/home/dani/github/WqSat_atcor/notebooks/../acolite/acolite/sentinel2/l1_convert.py:398: RuntimeWarning: Mean of empty slice
  ave_vza = np.nanmean(ave_vza, axis=2)
/home/dani/github/WqSat_atcor/notebooks/../acolite/acolite/sentinel2/l1_convert.py:399: RuntimeWarning: Mean of empty slice
  ave_vaa = np.nanmean(ave_vaa, axis=2)


Running acolite for /media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L1R.nc
Getting ancillary data for 2023-09-24T10:50:31.955037+00:00 1.402E 38.880N
EARTHDATA user name and password required for download of https://oceandata.sci.gsfc.nasa.gov/ob/getfile/GMAO_MERRA2.20230924T100000.MET.nc
Finished downloading file GMAO_MERRA2.20230924T100000.MET.nc
EARTHDATA user name and password required for download of https://oceandata.sci.gsfc.nasa.gov/ob/getfile/GMAO_MERRA2.20230924T110000.MET.nc
Finished downloading file GMAO_MERRA2.20230924T110000.MET.nc
No ozone file found for 2023-09-24T10:50:31.955037+00:00
No NCEP files found for 2023-09-24T10:50:31.955037+00:00
default uoz: 0.30 uwv: 1.50 pressure: 1013.25
current uoz: 0.30 uwv: 1.50 pressure: 1013.25
Using DSF atmospheric correction
Wrote lon (6119, 4840)
Wrote lat (6119, 4840)
Wrote sza (6119, 4840)
Wrote vza (6119, 4840)
Wrote raa (6119, 4840)
Loading LUTs ['ACOLITE-LUT-202110-MOD1', 'ACOLITE-LUT-202110-M

Warning 1: for band 1, nodata value has been rounded to -2147483648, Int32 being an integer datatype.


Wrote /media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L2W_l2_flags.tif
Wrote /media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L2W_chl_oc3.tif
Deleting /media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L1R.nc
Deleting /media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L2R.nc
Deleting /media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L2W.nc


{0: {'input': ['/media/dani/Seagate Basic/data/tests/S2B_MSIL1C_20230924T103659_N0509_R008_T31SCC_20230924T141949.SAFE',
   '/media/dani/Seagate Basic/data/tests/S2B_MSIL1C_20230924T103659_N0509_R008_T31SCD_20230924T141949.SAFE'],
  'l1r': ['/media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L1R.nc'],
  'l2r': ['/media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L2R.nc'],
  'l2w': ['/media/dani/Seagate Basic/data/tests/ACOLITE/S2B_MSI_2023_09_24_10_50_31_merged_L2W.nc']}}

In [3]:
# Ejemplo de uso (ajusta rutas):
input_folder = '/media/dani/Seagate Basic/data/tests/ACOLITE'
stack_and_clean(input_folder, '*L2W*.tif', os.path.join(input_folder, 'merged_L2W_Rrs.tif'))
stack_and_clean(input_folder, '*L2R_rhos*.tif', os.path.join(input_folder, 'merged_L2R_rhos.tif'))
stack_and_clean(input_folder, '*L2R_rhot*.tif', os.path.join(input_folder, 'merged_L2R_rhot.tif'))

/media/dani/Seagate Basic/data/tests/ACOLITE/merged_L2W_Rrs.tif creado con bandas nombradas (['None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk']) y 13 archivos eliminados.
/media/dani/Seagate Basic/data/tests/ACOLITE/merged_L2R_rhos.tif creado con bandas nombradas (['None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk']) y 11 archivos eliminados.
/media/dani/Seagate Basic/data/tests/ACOLITE/merged_L2R_rhot.tif creado con bandas nombradas (['None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk', 'None_unk']) y 13 archivos eliminados.
